In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amlc-csv-data/sample_test.csv
/kaggle/input/amlc-csv-data/sample_test_out.csv
/kaggle/input/amlc-csv-data/train.csv
/kaggle/input/amlc-csv-data/test.csv


/kaggle/input/amlc-csv-data/sample_test.csv
/kaggle/input/amlc-csv-data/sample_test_out.csv
/kaggle/input/amlc-csv-data/train.csv
/kaggle/input/amlc-csv-data/test.csv

In [11]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
train = pd.read_csv('/kaggle/input/amlc-csv-data/train.csv')

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sample_id        75000 non-null  int64  
 1   catalog_content  75000 non-null  object 
 2   image_link       75000 non-null  object 
 3   price            75000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB


In [15]:
# train['catalog_content']
"""
isme hyperparameter tuning karenge - karna chahiye
for both tfidf and xgb - par time lagega if tfidf me zyada ngram use karenge
and also data is very big
"""
xgb_text_only = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('xgb', xgb.XGBRegressor())
])

In [16]:
# xgb_text_only.fit(train['catalog_content'], train['price'])
X_train_text = train['catalog_content']
y_train_text = train['price']

In [9]:
xgb_text_only.fit(X_train_text, y_train_text)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [17]:
xgb.__version__

'2.0.3'

In [18]:
# SMAPE score 
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

In [19]:
smape_scorer = make_scorer(smape, greater_is_better=False)

In [14]:
y_pred = xgb_text_only.predict(X_train_text)

In [1]:
xgb_smape_score = smape(y_train_text, y_pred)
xgb_smape_score

NameError: name 'smape' is not defined

In [ ]:
# from lightgbm import LGBMRegressor

In [4]:
import lightgbm as lgbm

In [5]:
lgbm.__version__

'4.3.0'

In [ ]:
# import tensorflow as tf
# tf.__version__

2025-10-11 10:11:55.601652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760177515.879472     116 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760177515.958205     116 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'2.18.0'

In [20]:
lgbm_text_only = Pipeline([
    ('tfidf' , TfidfVectorizer(ngram_range=(1,2))),
    ('lgbm', lgbm.LGBMRegressor())
])

In [21]:
lgbm_text_only.fit(X_train_text, y_train_text)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 112.619506 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2353074
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 65515
[LightGBM] [Info] Start training from score 23.647654


Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('lgbm', LGBMRegressor())])

In [22]:
y_pred_lgbm = lgbm_text_only.predict(X_train_text)

In [23]:
lgbm_smape_score = smape(y_train_text, y_pred_lgbm)

In [26]:
lgbm_cross_val_score = -1 * cross_val_score(lgbm_text_only, X_train_text, y_train_text, cv=3, scoring=smape_scorer)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 59.218834 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1635503
[LightGBM] [Info] Number of data points in the train set: 50000, number of used features: 46329
[LightGBM] [Info] Start training from score 23.771094
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 56.169023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1628463
[LightGBM] [Info] Number of data points in the train set: 50000, number of used features: 46256
[LightGBM] [Info] Start training from score 23.530440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 55.677076 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM

In [27]:
lgbm_cross_val_score

array([62.39997319, 62.1155805 , 62.28571418])

```
will this appear as code ?
```